In [ ]:
#chat_saved_logistic_regressionmodel

In [5]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle

# Example dataset
data = pd.DataFrame({
    "income": [50000, 30000, 45000, 60000, 70000],
    "debt_to_income_ratio": [0.2, 0.5, 0.4, 0.1, 0.25],
    "credit_score": [700, 600, 650, 750, 720],
    "loan_amount": [20000, 15000, 25000, 10000, 20000],
    "default": [0, 1, 0, 0, 1]  # 0 = Non-default, 1 = Default
})

# FastAPI App
app = FastAPI()

# In-memory Logistic Regression model
model = LogisticRegression()
X = data.drop("default", axis=1)
y = data["default"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

# Define the input schema for retraining
class RetrainRequest(BaseModel):
    penalty: str = "l2"
    solver: str = "lbfgs"
    max_iter: int = 100
    test_size: float = 0.2

@app.post("/retrain/")
def retrain(request: RetrainRequest):
    global model, X_train, X_test, y_train, y_test
    try:
        # Retrain model with new parameters
        model = LogisticRegression(
            penalty=request.penalty,
            solver=request.solver,
            max_iter=request.max_iter
        )
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=request.test_size, random_state=42)
        model.fit(X_train, y_train)
        train_accuracy = accuracy_score(y_train, model.predict(X_train))
        test_accuracy = accuracy_score(y_test, model.predict(X_test))
        return {
            "message": "Model retrained successfully",
            "train_accuracy": train_accuracy,
            "test_accuracy": test_accuracy
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/predict/")
def predict(features: dict):
    try:
        feature_array = np.array([list(features.values())]).reshape(1, -1)
        prediction = model.predict(feature_array)[0]
        probability = model.predict_proba(feature_array)[0, 1]
        return {
            "prediction": int(prediction),
            "probability": float(probability)
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))


In [8]:
import openai
import requests

# Set up OpenAI API
openai.api_key = "your-openai-api-key"

# Backend URLs
BACKEND_URL = "http://127.0.0.1:8000"

def chat_with_user():
    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            break

        if "retrain" in user_input.lower():
            # Parse user input for retrain parameters
            params = {
                "penalty": "l2",
                "solver": "lbfgs",
                "max_iter": 200,
                "test_size": 0.3
            }  # Example: Adjust based on user input

            # Call backend retrain API
            response = requests.post(f"{BACKEND_URL}/retrain/", json=params)
            print("GPT:", response.json())

        elif "predict" in user_input.lower():
            # Example: Parse user features
            features = {
                "income": 55000,
                "debt_to_income_ratio": 0.3,
                "credit_score": 680,
                "loan_amount": 15000
            }  # Adjust based on user input

            # Call backend predict API
            response = requests.post(f"{BACKEND_URL}/predict/", json=features)
            print("GPT:", response.json())

        else:
            # Generic GPT response
            completion = openai.Completion.create(
                model="text-davinci-003",
                prompt=f"The user said: {user_input}. How should I respond?",
                max_tokens=150
            )
            print("GPT:", completion.choices[0].text.strip())

chat_with_user()
can you retain odel
If I give 

You:  


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
